In [ ]:
# Imports necessary libraries
import dropbox
import json
import random
import requests
import time
import os
import glob

from PIL import Image

In [ ]:
img_files = glob.glob("Data\\Images\\*.jpg")[:5]

In [ ]:
# Defines variables
folder_to_use = 'ARMS'
local_path = 'JSON_files\\' 
dropbox_path = '/CoralNet/' 

dropbox_token = ''
# Connects to our Dropbox account
dbx = dropbox.Dropbox(dropbox_token)
dbx.users_get_current_account()

In [ ]:
url     = 'https://coralnet.ucsd.edu/api/token_auth/'
payload = '{ "username" : "*", "password" : *"}'
headers = {"Content-type" : "application/vnd.api+json"}
coralnet_token = requests.post(url, data = payload, headers = headers).text.split(':"')[-1][:-2]

print(coralnet_token)

# The URL of the source used to annotate images
classifier_url = 'https://coralnet.ucsd.edu/api/classifier/12475/deploy/' # 12068, 12475

# Saves the classifier URL you will be using and your CoralNet authorization token
headers = {"Authorization": f"Token {coralnet_token}", 
           "Content-type": "application/vnd.api+json"}

In [ ]:
# Stores the image file names currently in dropbox folder
file_list = []

# Pulls out first N (limit) image files from folder
files = dbx.files_list_folder(f'{dropbox_path}{folder_to_use}', limit = 1)
file_list.extend(files.entries)

print("Number of files collected:", len(file_list))

In [ ]:
from coralnet_api_utils import *

width = 6000
height = 6000
offset = 1
percentage = .00056
sampling_method = 'grid'
per_image = 20

print("Number of points:", int(height * width * percentage * .01))
print("Offset:", int(2 * .01 * height))

In [ ]:
data = {"data":[]}

for _, entry in enumerate(file_list):
    
    assert entry.name == img_files[_].split("\\")[-1]
    
    img = Image.open(img_files[_])
    width, height = img.size
    
    for __ in range(per_image):
    
        # Local address and URL dropbox address to the image file, makes the URL shareable (swaps '0' with '1')
        local_address = f'{dropbox_path}{folder_to_use}/' + entry.name
        URL_address = dbx.sharing_create_shared_link(path = local_address, short_url = False, pending_upload = None)
        URL_address.url = URL_address.url[ : -1] + '1'

        points = get_points(height, width, offset, percentage, sampling_method)

        point_locations = {"type" : "image", 
                           "attributes": {"url" : URL_address.url, 
                                          "points": points
                                      }
                                  }

        data['data'].append(point_locations)

with open(f'{local_path}{folder_to_use}_Full_Request.json', 'w') as outfile:
    json.dump(data, outfile)

In [ ]:
len(data['data'])

In [ ]:
def check_status(r):
    
    # Sends a request to retrieve the completed annotations, obtains status update
    r_status = requests.get(url = 'https://coralnet.ucsd.edu' + r.headers['Location'], 
                            headers = {"Authorization": f"Token {coralnet_token}"})

    # Extracts the content from the status update
    curr_status, message = decode_status(r_status)
        
    return curr_status, message    

In [ ]:
export = {"data": []}

In [ ]:
patience = 60

for _ in range(len(data['data'])):
    
    reported = False
    
    # Break the data into batches of N (CoralNet requirement)
    current_batch = {"data" : data['data'][_ : _ + 1]}
    
    with open(f'{local_path}Batch_{str(_+1)}.json', 'w') as outfile:
        json.dump(current_batch, outfile)
    
    print("\nCurrently on batch:", str(_ + 1), " containing:", len(current_batch['data']), " entries.")

    r = requests.post(url = classifier_url, data = open(f"{local_path}Batch_{str(_+1)}.json"), headers = headers) 
    
    if(r.content.decode() != ''):
        print("Error: ", r)
        break
    else:
        print("Request sent successfully.")

    # Waits 5 seconds before attempting to retrieve results
    time.sleep(patience)     
    in_progress = True
    
    # Pings CoralNet every N seconds to check the status of the job
    while in_progress:
        
        curr_status, message = check_status(r)
        
        # Not complete yet, wait 60 seconds then ask again
        if message != '': 
            if(reported):
                print('.')
            else:
                print(message)
            reported = True
            time.sleep(patience) 
        
        # Is complete, store the annotations in export, close loop
        else: 
            print("Finished ", str(_ + 1), " batch" )
            export['data'].extend(curr_status['data'])
            in_progress = False


print("\nExporting annotations!")
# Creates a final json file with the relevant information included
with open(f'{local_path}{folder_to_use}_Exported_Annotations.json', 'w') as outfile:
      json.dump(export, outfile)

In [ ]:
print("\nExporting annotations!")

accumulative = open(f"{local_path}_Exported_Annotations.json")
accumulative['data'].extend(export)
    
# Creates a final json file with the relevant information included
with open(f'{local_path}{folder_to_use}_Exported_Annotations.json', 'w') as outfile:
      json.dump(accumulative, outfile)